In [ ]:
# @title Run this to get 10 config file
!pip install qrcode
import requests, json, csv, base64, base64, qrcode
from IPython.display import HTML, clear_output

%cd /content
!rm -r warp
!git clone https://github.com/epg900/warp.git
%cd warp
!chmod +x endip.sh && chmod +x warpendpoint
!./endip.sh ip4
f = open('result.csv', 'r')
csvdata = csv.DictReader(f)
data = [row for row in csvdata]
f.close()

%cd /content
clear_output()

r1 = requests.get("https://api.zeroteam.top/warp?format=sing-box")
r2 = requests.get("https://api.zeroteam.top/warp?format=sing-box")


j1 = json.loads(r1.text)
ipv6_1 = j1['local_address'][1]
pv1 = j1['private_key']
res1 = j1['reserved']

j2 = json.loads(r2.text)
ipv6_2 = j2['local_address'][1]
pv2 = j2['private_key']
res2 = j2['reserved']

n=1

for dat in data:
  ip1 = dat['IP:PORT'].split(":")[0]
  port1 = int(dat['IP:PORT'].split(":")[1])

  config =f'''{{
  "outbounds":
  [
    {{
      "type": "wireguard",
      "tag": "Warp-IR",
      "server": "{ip1}",
      "server_port": {port1},

      "local_address": [
        "172.16.0.2/32",
        "{ipv6_1}"
      ],
      "private_key": "{pv1}",
      "peer_public_key": "bmXOC+F1FxEMF9dyiK2H5/1SUtzH0JuVo51h2wPfgyo=",
      "reserved": {res1},

      "mtu": 1280,
      "fake_packets": "5-10"
    }},
    {{
      "type": "wireguard",
      "tag": "Warp-Main",
      "detour": "Warp-IR",
      "server": "{ip1}",
      "server_port": {port1},

      "local_address": [
          "172.16.0.2/32",
          "{ipv6_2}"
      ],
      "private_key": "{pv2}",
      "peer_public_key": "bmXOC+F1FxEMF9dyiK2H5/1SUtzH0JuVo51h2wPfgyo=",
      "reserved": {res2},

      "mtu": 1120,
      "fake_packets": "5-10"
    }}
  ]
}}
'''
  f = open(f'config{str(n)}.txt','w')
  f.write(config)
  f.close()
  n += 1
  if n > 10 :
    break

#curl -sSL https://gitlab.com/rwkgyg/CFwarp/raw/main/point/endip.sh -o endip.sh && chmod +x endip.sh && ./endip.sh
#pip install colab-xterm
#%load_ext colabxterm
#%xterm
conf=base64.b64encode(open("config4.txt","rb").read()).decode('utf-8')
config=f'data:text/html;base64,{conf}'

import qrcode
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr.add_data(config)
qr.make(fit=True)

img = qr.make_image(fill_color="black", back_color="white")
img.save("qrcode.png")

imgfile=base64.b64encode(open("qrcode.png","rb").read()).decode('ascii')

display(HTML("<center><img width='500px' height='500px'  src='data:image/png;base64,{}' /></center>".format(imgfile)))
html_text = '''<center>
                <input type="hidden" value="{}" id="clipborad-text">
                <button onclick="copyToClipboard()">Copy Config</button>
                <script>
                function copyToClipboard() {{


                    var copyText = document.getElementById("clipborad-text");
                    copyText.select();
                    navigator.clipboard.writeText(copyText.value);
                    alert("Copied the text: " + copyText.value);
                }}
                </script></center>'''.format(config,config)
display(HTML(html_text))